In [73]:
import numpy as np
from typing import Tuple
import tensorflow as tf
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.initializers import Constant
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [74]:
def get_dataset() -> Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]:
    (x_train, y_train), (x_test, y_test) = boston_housing.load_data()
    x_train = x_train.astype(np.float32)
    y_train = y_train.reshape((-1, 1)).astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_test = y_test.reshape((-1, 1)).astype(np.float32)

    return (x_train, y_train), (x_test, y_test)

In [75]:
def build_model(num_features: int, num_targets: int) -> Sequential:
    init_w = RandomUniform(minval=-1.0, maxval=1.0)
    init_b = Constant(value=0.0)

    model = Sequential()
    model.add(Dense(units=16, kernel_initializer=init_w, bias_initializer=init_b, input_shape=(num_features,))) 
        # Kernel waere die W-, bias die b-Matrix (Weights und Bias)
        # Müssen nicht angegeben werden, weil per Default praktische Zufallswerte gewählt werden
    model.add(Activation('relu'))
    model.add(Dense(units=num_targets))
    model.summary()
    return model

In [76]:
def r_squared(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
    error = tf.math.subtract(y_true, y_pred)
    squared_error = tf.math.square(error)
    numerator = tf.math.reduce_sum(squared_error)
    y_true_mean = tf.math.reduce_mean(y_true)
    mean_deviation = tf.math.subtract(y_true, y_true_mean)
    squared_mean_deviation = tf.math.square(mean_deviation)
    denominator = tf.reduce_sum(squared_mean_deviation)
    r2 = tf.math.subtract(1.0, tf.math.divide(numerator, denominator))
    r2_clipped = tf.clip_by_value(r2, clip_value_min=0.0, clip_value_max=1.0)
    return r2_clipped

In [77]:
def main() -> None:
    (x_train, y_train), (x_test, y_test) = get_dataset()

    print(f'x train shape: {x_train.shape}')
    print(f'y train shape: {y_train.shape}')
    print(f'x test shape: {x_test.shape}')
    print(f'y test shape: {y_test.shape}')
    
    num_features = 13
    num_targets = 1

    model = build_model(num_features, num_targets)

    model.compile(
        loss='mse',
        optimizer='Adam',
        metrics=[r_squared]
    )

    model.fit(
        x=x_train,
        y=y_train,
        epochs=5_000,
        batch_size=128,
        verbose=1,
        validation_data=(x_test, y_test)
    )

    score = model.evaluate(x=x_test, y=y_test, verbose=0)

In [78]:
if __name__=='__main__':
    main()

x train shape: (404, 13)
y train shape: (404, 1)
x test shape: (102, 13)
y test shape: (102, 1)
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 16)                224       
                                                                 
 activation_8 (Activation)   (None, 16)                0         
                                                                 
 dense_17 (Dense)            (None, 1)                 17        
                                                                 
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5000
4/4 [==============================] - 1s 106ms/step - loss: 131700.5469 - r_squared: 0.0000e+00 - val_loss: 133739.1719 - val_r_squared: 0.0000e+00
Epoch 2/5000
4/4 [==============================] - 0s 12ms/st